# Which Landolt (1992) standard stars are visible tonight?

Use `astroquery` to get a table of [Landolt (1992)](http://adsabs.harvard.edu/abs/1992AJ....104..340L) standard stars from Vizier. 

In [1]:
catalog_name = 'Landolt 1992'
observatory_name = 'Apache Point'

In [2]:
from astroquery.vizier import Vizier
from astropy.coordinates import SkyCoord
import astropy.units as u

catalog_list = Vizier.find_catalogs(catalog_name)
catalogs = Vizier.get_catalogs(catalog_list.keys())
Vizier.ROW_LIMIT = -1   # Otherwise would only show first 50 values
catalog_table = catalogs[0] # This is the table with the data

RAs = u.Quantity(catalog_table['_RAJ2000'].data, unit=u.deg)
Decs = u.Quantity(catalog_table['_DEJ2000'].data, unit=u.deg)
names = list(catalog_table['SimbadName'].data)

landolt_standards = SkyCoord(ra=RAs, dec=Decs)

/astro/users/bmmorris/.local/lib/python2.7/site-packages/astroquery/vizier/core.py:685: UserWarning: VOTABLE parsing raised exception: 
  warnings.warn("VOTABLE parsing raised exception: {0}".format(ex))


Set up an `Observer` and list of `FixedTarget`s in astroplan.

In [3]:
from astroplan import Observer, FixedTarget

obs = Observer.at_site(observatory_name)

target_list = [FixedTarget(coord=coord, name=name) 
               for coord, name in zip(landolt_standards, names)]

Determine which standards are observable tonight.

In [4]:
from astroplan import is_observable, observability_table, AltitudeConstraint, AtNightConstraint
from astropy.time import Time

constraints = [AltitudeConstraint(min=25*u.deg), 
               AtNightConstraint.twilight_astronomical()]

# Figure out when "tonight" is
present_time = Time.now()
if not obs.is_night(present_time):
    # If it's currently day time at runtime, find time of sunset and sunrise
    tonight_start = obs.sun_set_time(present_time, which='next')
    tonight_end = obs.sun_rise_time(present_time, which='next')
else:
    # Otherwise find time to next sunrise
    tonight_start = present_time
    tonight_end = obs.sun_rise_time(present_time, which='next')

table = observability_table(constraints, obs, target_list, 
                            time_range=Time([tonight_start, tonight_end]))
print(table)

 target name  ever observable always observable fraction of time observable
------------- --------------- ----------------- ---------------------------
[L92b] TPHE A           False             False                         0.0
[L92b] TPHE B           False             False                         0.0
[L92b] TPHE C           False             False                         0.0
[L92b] TPHE D           False             False                         0.0
[L92b] TPHE E           False             False                         0.0
[L92b] TPHE F           False             False                         0.0
[L92b] TPHE G           False             False                         0.0
   PG0029+024            True             False              0.344827586207
   PG0039+049            True             False              0.379310344828
    SA 92 309            True             False              0.379310344828
          ...             ...               ...                         ...
    SA 92 35